In [1]:
from lbmpy.session import *
from lbmpy.phasefield.analytical import *
import pystencils as ps

# Full analytical checks for N phase model

In [2]:
x = sp.symbols("x")
c_a = analytic_interface_profile(x)


def gamma(i, j):
    if i == j:
        return 0
    elif i < j:
        return sp.Symbol("gamma_%d%d" % (i, j))
    else:
        return sp.Symbol("gamma_%d%d" % (j, i))

The following checks if the 'tanh' shaped profile is a solution to $\mu_i = 0$ and if the excess free energy is the surface tension parameter.

In [3]:
numPhases = 3

print("Checking N =", numPhases)
c = sp.symbols("c_:%d" % (numPhases - 1,))
F = free_energy_functional_n_phases(order_parameters=c, surface_tensions=gamma)

μ = chemical_potentials_from_free_energy(F, c)

lastPhaseIdx = numPhases - 1

# Check all permutations of phases
for i in range(numPhases):
    for j in range(numPhases):
        if i == j:
            continue
        print("  -> Testing interface between", i, "and", j)
        substitutions = {c_i: 0 for c_i in c}
        if i != lastPhaseIdx:
            substitutions[c[i]] = c_a
        if j != lastPhaseIdx:
            substitutions[c[j]] = 1 - c_a

        for μ_i in μ:
            res = ps.fd.evaluate_diffs(μ_i.subs(substitutions), x).expand()
            assert res == 0, "Analytic interface profile wrong for phase between %d and %d" % (i, j)

        two_phase_free_energy = F.subs(substitutions)
        two_phase_free_energy = sp.simplify(ps.fd.evaluate_diffs(two_phase_free_energy, x))
        result = cosh_integral(two_phase_free_energy, x)
        assert result == gamma(i, j)

Checking N = 3
  -> Testing interface between 0 and 1
  -> Testing interface between 0 and 2
  -> Testing interface between 1 and 0
  -> Testing interface between 1 and 2
  -> Testing interface between 2 and 0
  -> Testing interface between 2 and 1
